In [1]:
import uproot
import vector
import numpy as np
import matplotlib.pyplot as plt
import awkward as ak
import fastjet

In [2]:
#Open a root file to play with
sample_file = uproot.open('root://xcache//store/group/lpcmetx/SIDM/ffNtupleV4/2018/SIDM_XXTo2ATo2Mu2E_mXX-100_mA-1p2_ctau-9p6_TuneCP5_13TeV-madgraph-pythia8/RunIIAutumn18DRPremix-102X_upgrade2018_realistic_v15-v1/210326_161703/0000/ffNtuple_1.root')
tree = sample_file['ffNtuplizer/ffNtuple']
#Load all the branches into memory (this is slow, and could be done in a 
#"lazy" way to only read the arrays that we end up using, but I'm too lazy to be lazy)
branches = tree.arrays()

In [3]:
#Define the jet clustering algorithm
jetdef = fastjet.JetDefinition(fastjet.antikt_algorithm, 0.4)

In [4]:
#Double check what information we have about the input candidates.
#These are what we will cluster into jets
tree.keys("ljsource*")

['ljsource_n',
 'ljsource_p4',
 'ljsource_p4/ljsource_p4.fCoordinates.fX',
 'ljsource_p4/ljsource_p4.fCoordinates.fY',
 'ljsource_p4/ljsource_p4.fCoordinates.fZ',
 'ljsource_p4/ljsource_p4.fCoordinates.fT',
 'ljsource_charge',
 'ljsource_type']

In [5]:
#Build the lj_sources into Vectors that fastjet can read
ljsource_p4 = vector.zip({"px": branches['ljsource_p4.fCoordinates.fX'], 
                      "py": branches['ljsource_p4.fCoordinates.fY'], 
                      "pz": branches['ljsource_p4.fCoordinates.fZ'], 
                      "t": branches['ljsource_p4.fCoordinates.fT'],
                       "particle_type":branches['ljsource_type']})

In [6]:
#Define the cluster sequence and actually do the clustering!
cluster = fastjet.ClusterSequence(ljsource_p4, jetdef)
jets = cluster.inclusive_jets()

#--------------------------------------------------------------------------
#                         FastJet release 3.4.0
#                 M. Cacciari, G.P. Salam and G. Soyez                  
#     A software package for jet finding and analysis at colliders      
#                           http://fastjet.fr                           
#	                                                                      
# Please cite EPJC72(2012)1896 [arXiv:1111.6097] if you use this package
# for scientific work and optionally PLB641(2006)57 [hep-ph/0512210].   
#                                                                       
# FastJet is provided without warranty under the GNU GPL v2 or higher.  
# It uses T. Chan's closest pair algorithm, S. Fortune's Voronoi code,
# CGAL and 3rd party plugin jet algorithms. See COPYING file for details.
#--------------------------------------------------------------------------


Compare results!

In [7]:
#Build pfjet vectors
#(Reminder that these are the lepton jets that were created by clustering
# when making the ntuples)
pfjet_p4 =  vector.zip({"px": branches['pfjet_p4.fCoordinates.fX'], 
                      "py": branches['pfjet_p4.fCoordinates.fY'], 
                      "pz": branches['pfjet_p4.fCoordinates.fZ'], 
                      "t": branches['pfjet_p4.fCoordinates.fT']}   )  
print("Original lepton-jet results!")
pfjet_p4.pt[:10,:].tolist()

Original lepton-jet results!


[[42.51500701904297],
 [],
 [],
 [67.18094635009766],
 [],
 [42.10684585571289],
 [],
 [46.05674362182617, 41.77437973022461],
 [52.83943557739258],
 [47.516212463378906]]

In [8]:
print("Our new jets!")
jets.pt[:10,:].tolist()

Our new jets!


[[42.51500699031592],
 [35.63738080195195],
 [18.131366540645228],
 [67.18094927131256],
 [33.83980956611488],
 [42.10684442200147],
 [25.417892405989903],
 [41.77437768749448, 46.05674447822695],
 [52.839434453098356],
 [47.51621299607016]]

I think any "missing" jets from the original collection come from the cuts we apply afterwards, primarily the pt > 30 GeV cut on lepton-jets and requiring muon-type lepton-jets to have an even number of candidates. (other cuts are eta < 2.4 and muon-type LJs must have a net zero charge)

-----------
Or comparing full p4:

In [9]:
print("Original lepton jets:")
for elem in pfjet_p4[:10,:]:
    print(elem)
print("Our new jets:")
for elem in jets[:10,:]:
    print(elem)

Original lepton jets:
[{x: -22.2, y: 36.2, z: 35.8, t: 55.6}]
[]
[]
[{x: -47.5, y: -47.6, z: -33.9, t: 75.3}]
[]
[{x: -13.9, y: -39.7, z: 47.7, t: 63.6}]
[]
[{x: -45.8, y: -5.23, z: 142, t: 150}, {x: 41.7, y: 2.1, z: 35.6, t: 54.9}]
[{x: -11.6, y: 51.6, z: -111, t: 123}]
[{x: 45, y: 15.3, z: 79.1, t: 92.3}]
Our new jets:
[{px: -22.2, py: 36.2, pz: 35.8, E: 55.6}]
[{px: 7.48, py: -34.8, pz: 21.2, E: 41.5}]
[{px: 14.2, py: -11.3, pz: -75.9, E: 78}]
[{px: -47.5, py: -47.6, pz: -33.9, E: 75.3}]
[{px: 18.8, py: -28.1, pz: -19.7, E: 39.1}]
[{px: -13.9, py: -39.7, pz: 47.7, E: 63.6}]
[{px: -19, py: -16.8, pz: -79.8, E: 83.8}]
[{px: 41.7, py: 2.1, pz: 35.6, E: 54.9}, {px: -45.8, py: -5.23, pz: 142, E: 150}]
[{px: -11.6, py: 51.6, pz: -111, E: 123}]
[{px: 45, py: 15.3, pz: 79.1, E: 92.3}]


---------------
Other tests:

In [10]:
#Double checking the type
ljsource_p4.type

18359 * var * Momentum4D["x": float32, "y": float32, "z": float32, "t": float32, "particle_type": int32]

In [11]:
#Looking at the input particles
ljsource_p4.particle_type[:10,:].tolist()

[[3, 3], [8], [3, 3], [3, 3], [3], [3, 8], [8, 8], [3, 3, 4], [3, 3], [3, 3]]

In [12]:
#Double checking the type
jets.type

18359 * var * Momentum4D["px": float64, "py": float64, "pz": float64, "E": float64]